In [2]:
%pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 501.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 26.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 14.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import boto3
import json
import base64
import time
import os
from botocore.config import Config
config = Config(
       connect_timeout=1000,
    read_timeout=1000,
)

session = boto3.session.Session(region_name='us-east-1')
client = session.client(service_name = 'bedrock-runtime', 
                                 config=config)


PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"

In [2]:
from json import JSONDecodeError
import re
def parse(text: str) -> str:
    pattern = r"<prompt>(.*?)</prompt>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        text = match.group(1)
        return text.strip()
    else:
        raise JSONDecodeError

In [3]:
with open("Amazon_Nova_Reel.pdf", "rb") as file:
    doc_bytes = file.read()

## Reel
- text 2 video

In [4]:
system_1 = \
"""
You are a Prompt Rewriting Expert for text-to-video models, with extensive knowledge in film and video production. 
You specialize in helping users improve their text prompts according to specific rules to achieve better model outputs, sometimes modifying the original intent if necessary. 

##You excel in the following areas:##
Comprehensive understanding of the world, physical laws, and various interactive video scenarios
Rich imagination to visualize perfect, visually striking video scenes from simple prompts
Extensive film industry expertise as a master director, capable of enhancing simple video descriptions with optimal cinematography and visual effects

##Your prompt rewriting should follow these guidelines:##
Prompting for video generation models differs from prompting for large language models (LLMs). 
Video generation models do not have the ability to reason or interpret explicit commands. 
Therefore, it's best to phrase your prompt as if it were an image caption or summary of the video rather than a command or conversation. 
You may want to include details about the subject, action, environment, lighting, style, and camera motion.
Subject: Add detailed characteristics of video subjects
Scene: Elaborate background details based on context
Emotional atmosphere: Describe the mood and overall ambiance
Cinematography: Specify shot types, camera angles, and perspectives (avoid complex camera movements)， Please refer to the guideline in DocumentPDFmessages.
Visual effects: Define style (e.g., Pixar, cinematic, hyperrealistic, 3D animation) and describe lighting, color tones, and contrast.


##Good Examples##
- Prompt: "Cinematic dolly shot of a juicy cheeseburger with melting cheese, fries, and a condensation-covered cola on a worn diner table. Natural lighting, visible steam and droplets. 4k, photorealistic, shallow depth of field"
- Prompt: "Arc shot on a salad with dressing, olives and other vegetables; 4k; Cinematic;"
- Prompt: "First person view of a motorcycle riding through the forest road."
- Prompt: "Closeup of a large seashell in the sand. Gentle waves flow around the shell. Camera zoom in."
- Prompt: "Clothes hanging on a thread to dry, windy; sunny day; 4k; Cinematic; highest quality;"
- Prompt: "Slow cam of a man middle age; 4k; Cinematic; in a sunny day; peaceful; highest quality; dolly in;"
- Prompt: "A mushroom drinking a cup of coffee while sitting on a couch, photorealistic."

##Ouput instruction##
Users may input prompts in Chinese or English, but your final output should be a single English paragraph not exceeding 90 words.
Put your reponse in <prompt></prompt>

"""

In [5]:
user_prompt = "In a nighttime forest, three glowing fireflies flutter about."

In [6]:
%%time
system = [
    {
        "text": system_1
    }
]

messages = [
    {
        "role": "user",
        "content": [
         {
            "document": {
                "format": "pdf",
                "name": "DocumentPDFmessages",
                "source": {
                    "bytes": doc_bytes
                }
            }
        },
         {"text": user_prompt},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"maxTokens": 2000, "topP": 0.9, "temperature": 0.3}


model_response = client.converse_stream(
    modelId=LITE_MODEL_ID, messages=messages, system=system, inferenceConfig=inf_params
)

text = ""
stream = model_response.get("stream")
if stream:
    for event in stream:
        if "contentBlockDelta" in event:
            text += event["contentBlockDelta"]["delta"]["text"]
            print(event["contentBlockDelta"]["delta"]["text"], end="")

<prompt>A nighttime forest scene with three glowing fireflies fluttering about, captured from a first-person aerial view. The forest is illuminated by the soft, natural light of the fireflies, creating a magical and serene atmosphere. The camera moves in slowly, enhancing the enchanting quality of the scene. Ultra HD, 8K resolution, photorealistic details.</prompt>CPU times: user 55.8 ms, sys: 16.2 ms, total: 72.1 ms
Wall time: 3.84 s


In [7]:
generated_prompt = parse(text)
print(len(generated_prompt))
print(generated_prompt)

350
A nighttime forest scene with three glowing fireflies fluttering about, captured from a first-person aerial view. The forest is illuminated by the soft, natural light of the fireflies, creating a magical and serene atmosphere. The camera moves in slowly, enhancing the enchanting quality of the scene. Ultra HD, 8K resolution, photorealistic details.


In [10]:
BUCKET = "s3://bedrock-video-generation-us-east-1-jlvyiv"
bedrock_runtime = session.client("bedrock-runtime")


def generate_video(bucket,text_prompt,input_image_base64=None):
    model_input = {
        "taskType": "TEXT_VIDEO",
        "textToVideoParams": {
            "text": generated_prompt,
        },
        "videoGenerationConfig": {
            "durationSeconds": 6,
            "fps": 24,
            "dimension": "1280x720",
            "seed": 0,  # Change the seed to get a different result
        },
    }
    if input_image_base64:
        model_input['textToVideoParams']['images'] =  [
            {
                "format": "png",
                "source": {
                    "bytes": input_image_base64
                }
            }
            ]

    try:
        # Start the asynchronous video generation job.
        invocation = bedrock_runtime.start_async_invoke(
            modelId="amazon.nova-reel-v1:0",
            modelInput=model_input,
            outputDataConfig={
                "s3OutputDataConfig": {
                    "s3Uri": BUCKET
                }
            }
        )
        return invocation

    except Exception as e:
        # Implement error handling here.
        message = e.response["Error"]["Message"]
        print(f"Error: {message}")
        return None

In [11]:
invocation = generate_video(bucket= BUCKET, text_prompt = generated_prompt)

while 1:
    response = bedrock_runtime.get_async_invoke(
        invocationArn=invocation['invocationArn']
    )
    status = response["status"]
    print(f"Status: {status}")
    time.sleep(10)
    if not status == 'InProgress':
        break

Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: Completed


In [18]:
import random
import string
def random_string_name(length=12):
        return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

def download_video_from_s3(s3_uri, local_path):
    """
    Download a video file from S3 to local storage
    
    Parameters:
    s3_uri (str): S3 URI in format 's3://bucket-name/path/to/video.mp4'
    local_path (str): Local path where the video will be saved
    """
    try:
        # Initialize S3 client
        s3_client = boto3.client('s3')
        
        # Parse S3 URI to get bucket and key
        if not s3_uri.startswith('s3://'):
            raise ValueError("Invalid S3 URI format. Must start with 's3://'")
        
        # Remove 's3://' and split into bucket and key
        path_parts = s3_uri[5:].split('/', 1)
        if len(path_parts) != 2:
            raise ValueError("Invalid S3 URI format")
        
        bucket_name = path_parts[0]
        s3_key = path_parts[1]
        
        # Create directory if it doesn't exist
        os.makedirs(local_path, exist_ok=True)
        fname = s3_key.split('/')[0]+'.mp4'
        # Download the file
        print(f"Downloading {s3_key} to {local_path}/{fname}")
        s3_client.download_file(bucket_name, s3_key, local_path+'/'+fname)
        print("Download completed successfully!")
        
        return f"{local_path}/{fname}"
        
    except Exception as e:
        print(f"Error downloading file: {str(e)}")
        return False


In [19]:
output_uri = response['outputDataConfig']['s3OutputDataConfig']['s3Uri']+'/output.mp4'
file_name = download_video_from_s3(output_uri,'./generated_videos')

Download completed successfully!


In [20]:
from IPython.display import Video, HTML
Video(file_name,width=1280, height=720)

## Image to Video

### 处理图片

In [55]:
from PIL import Image

def resize_image(image_path, output_path, target_width=1280, target_height=720):
    try:
        # 打开图片
        img = Image.open(image_path)
        
        # 获取原始图片尺寸
        original_width, original_height = img.size
        
        # 如果图片小于目标尺寸，直接返回原图
        if original_width <= target_width and original_height <= target_height:
            # img.save(output_path, quality=95)
            return
        
        # 计算目标宽高比
        target_ratio = target_width / target_height
        
        # 计算原始图片宽高比
        original_ratio = original_width / original_height
        
        # 裁剪图片以匹配目标比例
        if original_ratio > target_ratio:
            # 图片太宽，需要裁剪宽度
            new_width = int(original_height * target_ratio)
            left = (original_width - new_width) // 2
            img = img.crop((left, 0, left + new_width, original_height))
        elif original_ratio < target_ratio:
            # 图片太高，需要裁剪高度
            new_height = int(original_width / target_ratio)
            top = (original_height - new_height) // 2
            img = img.crop((0, top, original_width, top + new_height))
        
        # 调整图片大小到目标尺寸
        img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)
        
        # 保存处理后的图片
        img.save(output_path, quality=95)
        print(f"图片处理完成: {output_path}")
        
    except Exception as e:
        print(f"处理图片时出错: {str(e)}")


In [56]:
input_folder = "2"
output_folder = "output_image"

# 确保输出文件夹存在
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        resize_image(input_path, output_path)

图片处理完成: output_image/19.png
图片处理完成: output_image/7.png
图片处理完成: output_image/15.png
图片处理完成: output_image/8.png
图片处理完成: output_image/9.jpeg
图片处理完成: output_image/3.jpg
图片处理完成: output_image/18.png
图片处理完成: output_image/12.png
图片处理完成: output_image/10.png
图片处理完成: output_image/17.png


In [48]:
system_2 = \
"""
You are a Prompt rewriting expert for image-to-video models, with expertise in film industry knowledge and skilled at helping users output final text prompts based on input initial frame images and potentially accompanying text prompts. 
The main goal is to help other models produce better video outputs based on these prompts and initial frame images. Users may input only images or both an image and text prompt, where the text could be in Chinese or English.
Your final output should be a single paragraph of English prompt not exceeding 90 words.

##You are proficient in the knowledge mentioned in:##
-You have a comprehensive understanding of the world, knowing various physical laws and can envision video content showing interactions between all things.
-You are imaginative and can envision the most perfect, visually impactful video scenes based on user-input images and prompts.
-You possess extensive film industry knowledge as a master director, capable of supplementing the best cinematographic language and visual effects based on user-input images and simple descriptions.


##Please follow these guidelines for rewriting prompts:##
-Subject: Based on user-uploaded image content, describe the video subject's characteristics in detail, emphasizing details while adjusting according to user's text prompt.
-Scene: Detailed description of video background, including location, environment, setting, season, time, etc., emphasizing details.
-Emotion and Atmosphere: Description of emotions and overall atmosphere conveyed in the video, referencing the image and user's prompt.
Cinematography: Specify shot types, camera angles, and perspectives, Please refer to the guideline in DocumentPDFmessages.
-Visual Effects: Description of the visual style from user-uploaded images, such as Pixar animation, film style, realistic style, 3D animation, including descriptions of color schemes, lighting types, and contrast.

##Good Examples##
- Prompt: "Cinematic dolly shot of a juicy cheeseburger with melting cheese, fries, and a condensation-covered cola on a worn diner table. Natural lighting, visible steam and droplets. 4k, photorealistic, shallow depth of field"
- Prompt: "Arc shot on a salad with dressing, olives and other vegetables; 4k; Cinematic;"
- Prompt: "First person view of a motorcycle riding through the forest road."
- Prompt: "Closeup of a large seashell in the sand. Gentle waves flow around the shell. Camera zoom in."
- Prompt: "Clothes hanging on a thread to dry, windy; sunny day; 4k; Cinematic; highest quality;"
- Prompt: "Slow cam of a man middle age; 4k; Cinematic; in a sunny day; peaceful; highest quality; dolly in;"
- Prompt: "A mushroom drinking a cup of coffee while sitting on a couch, photorealistic."

##Ouput instruction##
Users may input prompts in Chinese or English, but your final output should be a single English paragraph not exceeding 90 words.
Put your reponse in <prompt></prompt>

"""

In [59]:
user_prompt = "冬季的树林中，红色的火车缓缓向前行驶."


In [63]:
%pip install python-magic

Note: you may need to restart the kernel to use updated packages.


In [64]:
import magic
def img_mime(image_path):
    try:
        mime = magic.Magic(mime=True)
        return mime.from_file(image_path)
    
    except Exception as e:
        print(f"python-magic detection error: {str(e)}")
        return None
            
with open(image_file, "rb") as f:
    image = f.read()

In [65]:
image_file = "output_image/3.jpg"
mime_type = img_mime(image_file)
mime_type

'image/jpeg'

In [67]:
%%time
system = [
    {
        "text": system_2
    }
]

messages = [
    {
        "role": "user",
        "content": [
         {
            "document": {
                "format": "pdf",
                "name": "DocumentPDFmessages",
                "source": {
                    "bytes": doc_bytes
                }
            }
        },
        {"image": {"format": mime_type.split('/')[1], "source": {"bytes": image}}},
         {"text": user_prompt},
        ],
    }
]

# Configure the inference parameters.
inf_params = {"maxTokens": 2000, "topP": 0.9, "temperature": 0.3}


model_response = client.converse_stream(
    modelId=LITE_MODEL_ID, messages=messages, system=system, inferenceConfig=inf_params
)

text = ""
stream = model_response.get("stream")
if stream:
    for event in stream:
        if "contentBlockDelta" in event:
            text += event["contentBlockDelta"]["delta"]["text"]
            print(event["contentBlockDelta"]["delta"]["text"], end="")

<prompt>A vintage red train slowly moves through a snowy forest in winter, with a monochromatic filter enhancing the serene atmosphere. The camera captures a first-person aerial view, dolly in shot, zooming in on the train as it glides along the tracks, surrounded by tall trees and a cloudy sky. Ultra HD, 8K resolution, photorealistic details.</prompt>CPU times: user 24.1 ms, sys: 8.08 ms, total: 32.2 ms
Wall time: 2.59 s


In [68]:
generated_prompt = parse(text)
print(len(generated_prompt))
print(generated_prompt)

337
A vintage red train slowly moves through a snowy forest in winter, with a monochromatic filter enhancing the serene atmosphere. The camera captures a first-person aerial view, dolly in shot, zooming in on the train as it glides along the tracks, surrounded by tall trees and a cloudy sky. Ultra HD, 8K resolution, photorealistic details.


In [71]:
with open(image_file, "rb") as f:
    image_bytes = f.read()
    image_base64 = base64.b64encode(image_bytes).decode("utf-8")
    
invocation = generate_video(bucket= BUCKET, text_prompt = generated_prompt,input_image_base64=image_base64)

while 1:
    response = bedrock_runtime.get_async_invoke(
        invocationArn=invocation['invocationArn']
    )
    status = response["status"]
    print(f"Status: {status}")
    time.sleep(10)
    if not status == 'InProgress':
        break

Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: InProgress
Status: Completed


In [72]:
output_uri = response['outputDataConfig']['s3OutputDataConfig']['s3Uri']+'/output.mp4'
file_name = download_video_from_s3(output_uri,'./generated_videos')

Download completed successfully!


In [73]:
Video(file_name,width=1280, height=720)